In [2]:
try:
    from google.colab import output
    output.enable_custom_widget_manager()
    print("Running in Google Colab")
except ModuleNotFoundError:
    print("Running locally – no Colab integration needed")

Running locally – no Colab integration needed


# A Simple Viscoelastic Fluid: The Maxwell Model

Many real materials — such as polymers, biological tissues, and asphalt — exhibit both *elastic* and *viscous* behavior.  
To capture this, we use **viscoelastic models** that combine a spring (elastic solid) and a dashpot (viscous fluid).

The **Maxwell model** is the simplest such model, consisting of a spring and a dashpot in **series**:

$$
\sigma + \lambda \dot{\sigma} = \eta \dot{\gamma}, \qquad \lambda = \frac{\eta}{G}
$$

Where:

- $ \sigma $ — shear stress  
- $ \dot{\gamma} $ — shear rate  
- $ G $ — elastic modulus (spring stiffness)  
- $ \eta $ — viscosity (dashpot resistance)  
- $ \lambda = \eta / G $ — relaxation time

In [3]:
from IPython.display import display, HTML

display(HTML("""
<script>
MathJax = {
  tex: {
    inlineMath: [['$', '$'], ['\\\\(', '\\\\)']],
    displayMath: [['$$', '$$'], ['\\\\[', '\\\\]']]
  }
};
</script>
"""))
print("✅ MathJax rendering enabled for both Jupyter and Colab.")

✅ MathJax rendering enabled for both Jupyter and Colab.


## Motivation for Creep and Stress Relaxation Tests

| Test | What we do | What we measure | What it tells us |
|------|-------------|-----------------|------------------|
| **Stress-relaxation** | Apply a sudden constant strain $ \gamma_0 $ | Observe $ \sigma(t) $ | Stress decays exponentially → gives relaxation time $ \lambda $ |
| **Creep** | Apply a sudden constant stress $ \sigma_0 $ | Observe $ \gamma(t) $ | Elastic jump + linear viscous flow → separates $ G $ and $ \eta $ |

**Analytical solutions**

1. **Stress relaxation:**

$$
\sigma(t) = G\,\gamma_0\,e^{-t/\lambda}
$$

2. **Creep:**

$$
\gamma(t) = \frac{\sigma_0}{G} + \frac{\sigma_0}{\eta}\,t
$$

The characteristic time $ \lambda = \eta / G $ governs the transition between elastic and viscous behavior.

In [4]:
# --- INTERACTIVE VISUALIZATION CELL ---

import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact

def stress_relaxation(t, G, eta, gamma0):
    lam = eta / G
    return G * gamma0 * np.exp(-t / lam)

def creep(t, G, eta, sigma0):
    return sigma0 / G + (sigma0 / eta) * t

@interact(
    G=widgets.FloatLogSlider(value=1e5, base=10, min=3, max=7, step=0.1, description='G [Pa]'),
    eta=widgets.FloatLogSlider(value=1e6, base=10, min=4, max=8, step=0.1, description='η [Pa·s]'),
    gamma0=widgets.FloatSlider(value=0.01, min=0.001, max=0.1, step=0.001, description='γ₀'),
    sigma0=widgets.FloatSlider(value=100, min=10, max=1000, step=10, description='σ₀ [Pa]'),
    T_process=widgets.FloatLogSlider(value=10, base=10, min=-1, max=3, step=0.1, description='T_process [s]'),
    tmax=widgets.FloatLogSlider(value=10, base=10, min=-2, max=4, step=0.1, description='t_max [s]')
)
def plot_maxwell(G, eta, gamma0, sigma0, T_process, tmax):
    lam = eta / G
    De = lam / T_process

    t = np.linspace(0, tmax, 400)
    sigma = stress_relaxation(t, G, eta, gamma0)
    gamma = creep(t, G, eta, sigma0)

    fig, ax = plt.subplots(1, 2, figsize=(10, 4))
    ax[0].plot(t, sigma, 'b', lw=2)
    ax[0].set_title('Stress Relaxation')
    ax[0].set_xlabel('Time [s]')
    ax[0].set_ylabel('Stress σ [Pa]')
    ax[0].grid(True)

    ax[1].plot(t, gamma, 'r', lw=2)
    ax[1].set_title('Creep')
    ax[1].set_xlabel('Time [s]')
    ax[1].set_ylabel('Strain γ [-]')
    ax[1].grid(True)

    plt.tight_layout()
    plt.show()

    print(f"Relaxation time λ = {lam:.3g} s")
    print(f"Deborah number De = {De:.3g}")

interactive(children=(FloatLogSlider(value=100000.0, description='G [Pa]', max=7.0, min=3.0), FloatLogSlider(v…